<a href="https://colab.research.google.com/github/shahriariit/Violence_Videos_Detection/blob/main/Violence_Detection_from_Video_Images_Part_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install moviepy

In [ ]:
from google.colab import drive
import cv2
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import csv
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score


In [ ]:
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/VDataset'

Mounted at /content/drive


In [ ]:
def load_images_from_folder(folder, label):
    """Load images from a folder, preprocess, and assign labels."""
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        # Load and preprocess image
        img = load_img(img_path, target_size=(224, 224))  # Resize image to 224x224
        img = img_to_array(img) / 255.0  # Normalize to [0, 1]
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)


def build_transfer_learning_model(base_model_fn, input_shape=(224, 224, 3), learning_rate=0.001):
    """
    Builds a binary classification model using a specified base model function.

    Parameters:
    - base_model_fn: a function from keras.applications (e.g., VGG16, ResNet50V2)
    - input_shape: input shape of the images
    - learning_rate: learning rate for the optimizer

    Returns:
    - Compiled Keras Model
    """
    # Load the base model without the top classification layer
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Corrected indentation for the evaluate function and its content
def evaluate(base_model, X_test, y_test, output_file="classification_metrics.csv"):
    # Predict probabilities

    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write the header row
        writer.writerow(["Accuracy", "Cohen-Kappa", "Precision", "Recall", "F1-Score", "AUROC", "MCC", "Specificity", "G-Mean", "Type I Error","Type II Error"])

        y_probs = base_model.predict(X_test)
        y_pred = (y_probs > 0.5).astype(int).flatten()

        # Classification Report (includes precision, recall, F1-score)
        print("Classification Report:\n", classification_report(y_test, y_pred))

        # Individual Metrics
        accuracy = accuracy_score(y_test, y_pred)
        cohen_kappa = cohen_kappa_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        auroc = roc_auc_score(y_test, y_probs, average='weighted')
        mcc = matthews_corrcoef(y_test, y_pred)
        specificity = specificity_score(y_test, y_pred, average='weighted')
        g_mean = geometric_mean_score(y_test, y_pred, average='weighted')
        type_i_error = 1 - g_mean
        type_ii_error = 1 - specificity


        writer.writerow([accuracy, cohen_kappa, precision, recall, f1, auroc, mcc, specificity, g_mean, type_i_error, type_ii_error])

        # Display results
        print("Accuracy:", accuracy)
        print("Cohen's Kappa:", cohen_kappa)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1-Score:", f1)
        print("AUROC:", auroc)
        print("Matthews Correlation Coefficient (MCC):", mcc)
        print("Specificity",specificity)
        print("Geometric Mean",g_mean)
        print("Type I Error",type_i_error)
        print("Type II Error",type_ii_error)

In [ ]:
# Paths to the frames
nonviolence_folder = '/content/drive/MyDrive/VDataset/Non_Violence'
violence_folder = '/content/drive/MyDrive/VDataset/Violence'

In [ ]:
# Load images and labels for each class
violence_images, violence_labels = load_images_from_folder(violence_folder, 1)  # Label 1 for Violence
nonviolence_images, nonviolence_labels = load_images_from_folder(nonviolence_folder, 0)  # Label 0 for Non-Violence

# Combine the datasets
X = np.concatenate((violence_images, nonviolence_images), axis=0)
y = np.concatenate((violence_labels, nonviolence_labels), axis=0)

# Shuffle the dataset
X, y = shuffle(X, y, random_state=42)

print(f"Dataset loaded and shuffled.")
print(f"Total samples: {len(y)}")
print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")

Dataset loaded and shuffled.
Total samples: 2474
Shape of X: (2474, 224, 224, 3), Shape of y: (2474,)


In [ ]:
# Split dataset into training and testin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_densenet169 = build_transfer_learning_model(DenseNet169)
model_densenet169.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet169 = model_densenet169.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet169}")
evaluate(model_densenet169, X_test, y_test,"densenet169_classification.csv")

51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 116s 1s/step - accuracy: 0.7585 - loss: 0.6947 - val_accuracy: 0.9455 - val_loss: 0.1205
Epoch 2/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 135ms/step - accuracy: 0.9642 - loss: 0.1161 - val_accuracy: 0.9596 - val_loss: 0.0900
Epoch 3/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 119ms/step - accuracy: 0.9657 - loss: 0.0899 - val_accuracy: 0.9434 - val_loss: 0.1119
Epoch 4/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - accuracy: 0.9745 - loss: 0.0575 - val_accuracy: 0.9636 - val_loss: 0.0862
Epoch 5/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 11s 137ms/step - accuracy: 0.9800 - loss: 0.0561 - val_accuracy: 0.9636 - val_loss: 0.0759
Epoch 6/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 118ms/step - accuracy: 0.9865 - loss: 0.0379 - val_accuracy: 0.9737 - val_loss: 0.0667
Accuracy: 0.973737359046936
16/16 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.96      

In [ ]:
model_densenet201 = build_transfer_learning_model(DenseNet201)
model_densenet201.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet201 = model_densenet201.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet201}")
evaluate(model_densenet201, X_test, y_test,"densenet201_classification.csv")

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 118s 1s/step - accuracy: 0.7752 - loss: 0.7001 - val_accuracy: 0.9475 - val_loss: 0.1233
Epoch 2/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 56s 149ms/step - accuracy: 0.9557 - loss: 0.1165 - val_accuracy: 0.9636 - val_loss: 0.0931
Epoch 3/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 145ms/step - accuracy: 0.9701 - loss: 0.0753 - val_accuracy: 0.9677 - val_loss: 0.0791
Epoch 4/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 11s 161ms/step - accuracy: 0.9871 - loss: 0.0401 - val_accuracy: 0.9636 - val_loss: 0.0939
Epoch 5/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 148ms/step - accuracy: 0.9897 - loss: 0.0383 - val_accuracy: 0.9737 - val_loss: 0.0809
Epoch 6/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 11s 161ms/step - accuracy: 0.9933 - loss: 0.0214 - val_accuracy: 0.9798 - val_loss: 0.0806
Accuracy: 0.9797979593276978
16/16 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97    

In [ ]:
model_inceptionv3 = build_transfer_learning_model(InceptionV3)
model_inceptionv3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_inceptionv3 = model_inceptionv3.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_inceptionv3}")
evaluate(model_inceptionv3, X_test, y_test,"inceptionv3_classification.csv")

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
Epoch 1/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 42s 420ms/step - accuracy: 0.7214 - loss: 0.8518 - val_accuracy: 0.9172 - val_loss: 0.2131
Epoch 2/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 11s 82ms/step - accuracy: 0.9139 - loss: 0.2043 - val_accuracy: 0.9212 - val_loss: 0.1669
Epoch 3/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 86ms/step - accuracy: 0.9521 - loss: 0.1277 - val_accuracy: 0.9152 - val_loss: 0.2108
Epoch 4/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.9537 - loss: 0.1199 - val_accuracy: 0.9212 - val_loss: 0.1614
Epoch 5/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.9660 - loss: 0.0978 - val_accuracy: 0.9374 - val_loss: 0.1499
Epoch 6/6
62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.9721 - loss: 0.0776 - val_accuracy: 0.9455 - val_loss: 0.1230
Accuracy: 0.9454545378684998
16/16 ━━━━━━━━━━━━━━━━━━━━ 13s 485ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.95     